In [1]:
import os
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from PIL import Image

In [2]:
# Dataset Path (adjust these paths based on your Kaggle dataset structure)
#zip_path = '/kaggle/input/garbage-images.zip'  # Corrected zip_path and added .zip extension
extract_path = '/kaggle/input/garbage-images' #Where to extract to.

# Extract the zip file
#with zipfile.ZipFile(zip_path, 'r') as zip_ref: # Corrected variable name
#    zip_ref.extractall(extract_path)

# Correct the paths to the extracted directories
train_dir = os.path.join(extract_path, 'filtered_DATASET', 'TRAIN') #adjust 'train' if needed
test_dir = os.path.join(extract_path, 'filtered_DATASET', 'TEST') #adjust 'test' if needed

# Define Transforms
transform_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Custom Dataset Class
class WasteDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = os.listdir(root_dir)
        self.image_paths = []
        self.labels = []

        for label, class_name in enumerate(self.classes):
            class_dir = os.path.join(root_dir, class_name)
            for image_name in os.listdir(class_dir):
                self.image_paths.append(os.path.join(class_dir, image_name))
                self.labels.append(label)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

# Create Datasets and DataLoaders
train_dataset = WasteDataset(train_dir, transform=transform_train)
test_dataset = WasteDataset(test_dir, transform=transform_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# EfficientNetV2 Model (simplified, for demonstration)
class EfficientNetV2Lite(nn.Module):
    def __init__(self, num_classes=2):
        super(EfficientNetV2Lite, self).__init__()
        self.features = torchvision.models.efficientnet_v2_s(pretrained=True).features
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(1280, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

# Initialize Model, Loss Function, and Optimizer
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = EfficientNetV2Lite().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training Loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

# Evaluation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on test set: {100 * correct / total}%")

# Saving model to Kaggle's working directory
torch.save(model.state_dict(), '/kaggle/working/waste_classification_model.pth')

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_V2_S_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_V2_S_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_v2_s-dd5fe13b.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_v2_s-dd5fe13b.pth
100%|██████████| 82.7M/82.7M [00:00<00:00, 205MB/s] 


Epoch 1/10, Loss: 0.25075830444268427
Epoch 2/10, Loss: 0.20541344153642996
Epoch 3/10, Loss: 0.1865541626607598
Epoch 4/10, Loss: 0.1691878385198822
Epoch 5/10, Loss: 0.1575725225468675
Epoch 6/10, Loss: 0.14068830512814787
Epoch 7/10, Loss: 0.13099571716003172
Epoch 8/10, Loss: 0.11703787733283898
Epoch 9/10, Loss: 0.10996147847406236
Epoch 10/10, Loss: 0.09270563891779275
Accuracy on test set: 91.70323095333067%
